## PyTorch implementation of the LSTM

Now that we know how the LSTM cell works, let's see how easy it is to use in PyTorch!

Definition of our LSTM network. We define a LSTM layer using the [nn.LSTM](https://pytorch.org/docs/stable/nn.html#lstm) class. The LSTM layer takes as argument the size of the input and the size of the hidden state like in our Nanograd implementation.

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cebrina's   Path: C:\Users\cebri\Documents\Wind_Power_Estimation\Data
# Guillermo's Path: C:\DTU\\02456 - Deep Learning\Project\Datasets
# Tomi's      Path: C:\Users\PC\Documents\GitHub\WindPower_Estimation

dataPath = r"C:\Users\cebri\Documents\Wind_Power_Estimation\Data"
train1_dataset = pd.read_csv (r''+dataPath+'\Case1\Dataset_Train_1.csv', header=0, delim_whitespace=False)
train2_dataset = pd.read_csv (r''+dataPath+'\Case2\Dataset_Train_2.csv', header=0, delim_whitespace=False)
train3_dataset = pd.read_csv (r''+dataPath+'\Case3\Dataset_Train_3.csv', header=0, delim_whitespace=False)
test1_dataset = pd.read_csv (r''+dataPath+'\Case1\Dataset_Test_1.csv', header=0, delim_whitespace=False)
test2_dataset = pd.read_csv (r''+dataPath+'\Case2\Dataset_Test_2.csv', header=0, delim_whitespace=False)
test3_dataset = pd.read_csv (r''+dataPath+'\Case3\Dataset_Test_3.csv', header=0, delim_whitespace=False)

print( test1_dataset ) 

print(type(test1_dataset))


                 Date_Time  Direction_10m  Speed_10m  Temperature_10m  \
0      2020-05-18 13:15:00       0.233983   0.523846         0.708869   
1      2020-05-18 13:30:00       0.236769   0.527692         0.706422   
2      2020-05-18 13:45:00       0.242340   0.530000         0.703976   
3      2020-05-18 14:00:00       0.247911   0.530769         0.701835   
4      2020-05-18 14:15:00       0.250696   0.530000         0.699847   
...                    ...            ...        ...              ...   
29436  2021-01-25 18:45:00       0.370474   0.176154         0.463456   
29437  2021-01-25 19:00:00       0.370474   0.176154         0.463456   
29438  2021-01-25 19:15:00       0.370474   0.176154         0.463456   
29439  2021-01-25 19:30:00       0.370474   0.176154         0.463456   
29440  2021-01-25 19:45:00       0.370474   0.176154         0.463456   

       Pressure_seaLevel  Air_Density_10m  Direction_50m  Speed_50m  \
0               0.512525         0.250737       0.23

In [ ]:
#RNN

In [4]:
# Definition of class for batch generation
from torch.utils.data import Dataset, DataLoader
class TimeSeriesDataSet(Dataset):
  """
  This is a custom dataset class. It can get more complex than this, but simplified so you can understand what's happening here without
  getting bogged down by the preprocessing
  """
  def __init__(self, X, Y, Z):
    self.X = X
    self.Y = Y
    self.Z = Z
    if len(self.X) != len(self.Y) :
      raise Exception("The length of X does not match the length of Y")
    if len(self.X) != len(self.Z) :
      raise Exception("The length of X does not match the length of Z")
    if len(self.Z) != len(self.Y) :
      raise Exception("The length of Z does not match the length of Y")

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    # note that this isn't randomly selecting. It's a simple get a single item that represents an x and y
    _x = self.X[index]
    _y = self.Y[index]
    _z = self.Z[index]

    return _x , _y , _z 

In [7]:
class MyRecurrentNet(nn.Module):
    def __init__(self):
        super(MyRecurrentNet, self).__init__()
        
        # Recurrent layer
        self.lstm = nn.LSTM(1, 50)
        
        # Output layer
        self.l_out = nn.Linear(in_features=50,
                            out_features=1,
                            bias=False)
        
    def forward(self, x):
        # RNN returns output and last hidden state
        x, (h, c) = self.lstm(x)
        
        # Flatten output for feed-forward layer
        x = x.view(-1, self.lstm.hidden_size)
        
        # Output layer
        x = self.l_out(x)
        
        return x

In [28]:
#Data for the first testset
validation_set_park_power = test1_dataset[["Park_Power_[KW]"]]
training_set_park_power = train1_dataset[["Park_Power_[KW]"]]

def generate_time_lags(df, n_lags):
    df_n = df.copy()
    for n in range(1, n_lags + 1):
        df_n[f"lag{n}"] = df_n["Park_Power_[KW]"].shift(n)
    df_n = df_n.iloc[n_lags:]
    return df_n
    
input_dim = 100

validation_set_generated = generate_time_lags(validation_set_park_power, input_dim)
training_set_generated = generate_time_lags(training_set_park_power, input_dim)

from torch.utils.data import TensorDataset, DataLoader

batch_size = 20

train_loader = DataLoader(training_set_generated, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(validation_set_generated, batch_size=batch_size, shuffle=False, drop_last=True)

C:\Users\cebri\AppData\Local\Temp/ipykernel_11844/2033497344.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_n[f"lag{n}"] = df_n["Park_Power_[KW]"].shift(n)


In [29]:
# Hyper-parameters
num_epochs = 200

# Initialize a new network
net = MyRecurrentNet()

# Define a loss function and optimizer for this problem
# YOUR CODE HERE!
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

# Track loss
training_loss, validation_loss = [], []

# For each epoch
for i in range(num_epochs):
    
    # Track loss
    epoch_training_loss = 0
    epoch_validation_loss = 0
    
    net.train()
    
     # For each sentence in training set
    for inputs, targets in train_loader:
              
        # Forward pass
        outputs = net.forward(inputs_one_hot)
        
        # Compute loss
        # YOUR CODE HERE!
        loss = criterion(outputs, targets_idx)
        
        # Backward pass
        # zero grad, backward, step...
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update loss
        epoch_training_loss += loss.detach().numpy()
    
    net.eval()
        
    # For each sentence in validation set
    for inputs, targets in test_loader:
                        
        # Forward pass
        outputs = net.forward(inputs_one_hot)
        
        # Compute loss
        loss = criterion(outputs, targets_idx)
        
        # Update loss
        epoch_validation_loss += loss.detach().numpy()

        
    # Save loss for plot
    training_loss.append(epoch_training_loss/len(training_set))
    validation_loss.append(epoch_validation_loss/len(validation_set))

    # Print loss every 10 epochs
    if i % 10 == 0:
        print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')

        
# Plot training and validation loss
epoch = np.arange(len(training_loss))
plt.figure()
plt.plot(epoch, training_loss, 'r', label='Training loss',)
plt.plot(epoch, validation_loss, 'b', label='Validation loss')
plt.legend()
plt.xlabel('Epoch'), plt.ylabel('NLL')
plt.show()

KeyError: 0